Goal :

Create an Agent with the following tools

- Question answering Retrieval from Pinecone VDB
- Retrieving instructions for a given task from the pre-existing VDB
- Searching in a Google Drive folder for files

Agent's function is to fullfill the user's request: question OR generation task


In [1]:
import os 
from dotenv import load_dotenv 
load_dotenv()
from langchain_community.document_loaders.mongodb import MongodbLoader
from langchain_pinecone import PineconeVectorStore
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.document_loaders import S3FileLoader
from langchain_core.documents import Document
import logging
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
import streamlit as st
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import TextLoader

def get_pinecone_vector_store(embeddings_model,namespace="default",index_name: str = os.getenv("PINECONE_INDEX_NAME")) -> PineconeVectorStore:
    try:
        # Initialize Pinecone client
        pinecone_api_key = os.getenv("PINECONE_API_KEY")
        if not pinecone_api_key:
            raise ValueError("Pinecone API key is not set. Please set it in the environment variables.")
        
        pc = Pinecone(api_key=pinecone_api_key)

        # Check if the index exists
        existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
        if index_name in existing_indexes:
            print(f"Index '{index_name}' already exists. Retrieving it...")
            index = pc.Index(index_name)
        else:
            print(f"Index '{index_name}' does not exist. Please create one")


        # Initialize and return the vector store
        vector_store = PineconeVectorStore(index=index, embedding=embeddings_model,namespace=namespace)
        return vector_store

    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    

/home/asmaa/work/BV_Lab/poc-env/lib/python3.10/site-packages/langchain_pinecone/__init__.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.embeddings import PineconeEmbeddings
/home/asmaa/work/BV_Lab/poc-env/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
import os 
from langchain_community.agent_toolkits import GmailToolkit
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

# Instantiate GmailToolkit
toolkit = GmailToolkit()

# Get tools
google_tools = toolkit.get_tools()



In [3]:
google_tools

[GmailCreateDraft(api_resource=<googleapiclient.discovery.Resource object at 0x7f5f483fce50>),
 GmailSendMessage(api_resource=<googleapiclient.discovery.Resource object at 0x7f5f483fce50>),
 GmailSearch(api_resource=<googleapiclient.discovery.Resource object at 0x7f5f483fce50>),
 GmailGetMessage(api_resource=<googleapiclient.discovery.Resource object at 0x7f5f483fce50>),
 GmailGetThread(api_resource=<googleapiclient.discovery.Resource object at 0x7f5f483fce50>)]

In [3]:
from langchain_googldrive.tools.google_drive.tool import GoogleDriveSearchTool
from langchain_googldrive.utilities.google_drive import GoogleDriveAPIWrapper

folder_id = ""
# By default, search only in the filename.
tool = GoogleDriveSearchTool(
    api_wrapper=GoogleDriveAPIWrapper(
        folder_id=folder_id,
        num_results=2,
        template="gdrive-query-in-folder",  # Search in the body of documents
    )
)

ModuleNotFoundError: No module named 'langchain_googldrive'

In [15]:
# Retrieval tool
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
import os 
from langgraph.checkpoint.memory import MemorySaver
from langchain.tools.retriever import create_retriever_tool
from langgraph.prebuilt import create_react_agent
from dotenv import load_dotenv 
load_dotenv()
memory = MemorySaver()

vector_store = get_pinecone_vector_store(OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"),model="text-embedding-3-large"))
docs_retriever = vector_store.as_retriever()

inst_vector_store = get_pinecone_vector_store(OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"),model="text-embedding-3-large"),index_name="instructions-index")
inst_retriever = vector_store.as_retriever()

llm = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=openai_api_key)

docs_retrieval_tool = create_retriever_tool(
    docs_retriever,
    "answer_question",
    "Invoked always when a user asks a question."
)

instructions_retrieval_tool = create_retriever_tool(
    inst_retriever,
    "instructions_retriever",
    "Used to retrieve relevant instructions on how to complete a task when the user asks you to perform a certain task like letter writting, email drafting etc."
)
tools = [docs_retrieval_tool,instructions_retrieval_tool] + google_tools

# ReAct Agent Exec
react_agent_executor = create_react_agent(llm, tools, checkpointer=memory)

# OpenAI functions agent
instructions = "You are an assistant."
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)
llm = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=openai_api_key)
openai_agent = create_openai_functions_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=openai_agent,
    tools=tools,
    verbose=True
)

Index 'poc-vessel-manag' already exists. Retrieving it...
Index 'instructions-index' already exists. Retrieving it...


/home/asmaa/work/BV_Lab/poc-env/lib/python3.10/site-packages/langsmith/client.py:312: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [18]:
out = agent_executor.invoke(    {
        "input": "draft an email to asmaahadir11@gmail.com",
        # "chat_history": [
        #     HumanMessage(content="hi! my name is bob"),
        #     AIMessage(content="Hello Bob! How can I assist you today?"),
        # ],
    }
    )

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Could you please provide the subject and the message you would like to include in the email?

> Finished chain.


In [19]:
out

{'input': 'draft an email to asmaahadir11@gmail.com',
 'output': 'Could you please provide the subject and the message you would like to include in the email?'}

Agent now has acce